### An illustration of personal meeting with Dr. Solodkin at Jan 03

1. generate the graph, ggdist
2. use script to calculate the meta-connectivity of cluster 3.

In [1]:
#!/usr/bin/python
import warnings
warnings.filterwarnings('ignore')
from turtle import color
import pandas as pd
import numpy as np
import sys
sys.path.append('/home/yat-lok/workspace/Network-science-Toolbox/Python')
# sys.path.append('/home/wayne/github/TVB_workflow/new_g_optimal')
# sys.path.append('/home/wayne/github/Network-science-Toolbox/Python')
from TS2dFCstream import TS2dFCstream
from dFCstream2Trimers import dFCstream2Trimers
from dFCstream2MC import dFCstream2MC
import logging
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats.stats import pearsonr
import itertools
import os
import scipy.io
import seaborn as sns
import matplotlib.pyplot as plt

"""
@Author: Yile Wang

This script is used to calculate the homotopic meta-connectivity in four groups, SNC, NC, MCI, AD
"""


'\n@Author: Yile Wang\n\nThis script is used to calculate the homotopic meta-connectivity in four groups, SNC, NC, MCI, AD\n'

In [2]:
# brain region labels for your reference
regions = ['aCNG-L', 'aCNG-R','mCNG-L','mCNG-R','pCNG-L','pCNG-R', 'HIP-L','HIP-R','PHG-L','PHG-R','AMY-L','AMY-R', 'sTEMp-L','sTEMP-R','mTEMp-L','mTEMp-R']
regionswithgroups = ['groups','aCNG-L', 'aCNG-R','mCNG-L','mCNG-R','pCNG-L','pCNG-R', 'HIP-L','HIP-R','PHG-L','PHG-R','AMY-L','AMY-R', 'sTEMp-L','sTEMP-R','mTEMp-L','mTEMp-R']
group = ['SNC', 'NC', 'MCI','AD']

regions14 = []
for i in range(14):
    wt = ["regions_", str(i)]
    wt = "".join(wt)
    regions14.append(wt)

djouya_index = scipy.io.loadmat('/home/yat-lok/workspace/data4project/lateralization/gc1sec_res/meta/idx.mat')
mc_index = djouya_index['idx'].T[0] -1

path = '/home/yat-lok/workspace/data4project/lateralization/tvb_parameters.xlsx'
coData = pd.read_excel(path, index_col=0)
print(mc_index)

[  5   6   7   8   9  10  11  12  13  14  19  20  21  22  23  24  25  26
  27  28  32  33  34  35  36  37  38  39  40  41  44  45  46  47  48  49
  50  51  52  53  55  56  57  58  59  60  61  62  63  64  65  66  67  68
  69  70  71  72  73  74   0   1   2   3   4  15  16  17  18  29  30  31
  42  43  54  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119]


In [20]:
# iterate simulated functional connectivity
if __name__ == "__main__":
    tmp_col = ["group", "caseid"]
    pcng_vector = np.array([[*range(190)]])
    for grp in group:
        # subject case ids
        ldir = os.listdir("/home/yat-lok/workspace/data4project/lateralization/ts_fmri/"+ grp+'-TS')
        # ldir = os.listdir('/home/wayne/TS-4-Vik/'+grp+'-TS/')
        
        for ind, y in enumerate(ldir):
            # import empirical functional connectivity
            # Here is the path of the mat file of the FC data
            pth_efc = "/home/yat-lok/workspace/data4project/lateralization/ts_fmri/"+ grp+'-TS/'+ y +"/ROISignals_"+ y +".mat"
            # pth_efc = "/home/wayne/TS-4-Vik/"+grp+"-TS/"+ y +"/ROISignals_"+ y +".mat"
            ea = scipy.io.loadmat(pth_efc)
            all = ea['ROISignals']
            df = pd.DataFrame.from_dict(all)
            df.columns = regions
            # calculate the meta-connectivity, using existing script:
            dFCstream = TS2dFCstream(df.to_numpy(), 5, None, '2D')
            # Calculate MC
            MC_MC = dFCstream2MC(dFCstream)
            # sort the matrix
            mc = np.zeros((len(mc_index),len(mc_index)))
            for mc_row in range(len(mc_index)-1):
                for mc_col in range(mc_row+1,len(mc_index)):
                    mc[mc_row, mc_col] = MC_MC[mc_index[mc_row], mc_index[mc_col]]
            mc_sorted_cluster3 = mc + mc.T
            mc_cluster3 = mc_sorted_cluster3[40:60, 40:60]
            tmp_mc = np.tril(mc_cluster3,k=0).flatten()
            tmp_cluster3 = np.array([[y for y in tmp_mc if y != 0.0]])
            pcng_vector = np.concatenate((pcng_vector, tmp_cluster3), axis = 0)
    pd.DataFrame(pcng_vector).to_excel("../../data4project/lateralization/gc1sec_res/meta/mc_cluster3.xlsx")

[[  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
   18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
   36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
   54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
   72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
   90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
  108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
  126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143
  144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161
  162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179
  180 181 182 183 184 185 186 187 188 189]]
